In [ ]:
#import necessary python packages
import pandas as pd
import numpy as np
import sys
import seaborn as sns
sns.set_context('paper')

#provide the path to local folder containing python functions for pbmc analysis
sys.path.insert(1, '/home/siina/python_scripts/moncyte_analysis/analysis_tools/pbmc_analysis_tools/')
#import functions for pbmc analysis
from DataPrep import DataPrep
from NormaliseData import NormaliseData
from Plate import Plate
from DataCleanup import DataCleanup
from WellCalc import WellCalc
from DrawHeatmap import DrawHeatmap
from FoldCalc import FoldCalc

#provide paths to used folders, update these paths again I accidentally overwrote those. 
where = '/home/siina/python_scripts/moncyte_analysis/ME00079/'
layouts_folder = where+ 'layouts/' # where your csv layout file is located
results_folder = where+ 'cellp_results/' #where is your cellprofiler output folder located
output_folder = where+ 'out_results/' 

In [ ]:
pd.set_option('display.max_columns', None)
#provide the name of the cellprofiler output file
filename = 'ME00079_P1Cells.csv' 


file_loc = results_folder + filename
print('Data location: ',file_loc)

#read the cellprofiler output file and extract experiment and run name from the filename
sr_all = pd.read_csv(file_loc)
#print(sr_all)
exp_name = filename[:-10]
print('Experiment name: ',exp_name)
run_name = filename[0:6]
print('Run name: ', run_name)


experimentlayout = layouts_folder+exp_name+'_layout.csv'
print('Plate layout: ',experimentlayout)


#initialise the class for transforming the dataframe to readable format
my_frame = DataPrep(file_loc, exp_name)
print(my_frame)
#set the stain name for channel C
my_frame.set_stain_c('LD')
#print('Stain C = ', my_frame.stain_c)

#set the pixel size
my_frame.set_pixel_size(0.094916838)
#print('Pixel size = ', my_frame.pix_res)

#set the stain name for channel D
my_frame.set_stain_d('CD16')
#print('Stain C = ', my_frame.stain_c)

#set the cut_off
my_frame.set_cutoff(90)

dat = my_frame.org_calc(experimentlayout, 384)
print(dat.head(2))
#remove outliers on a single-cell level
clean_data = DataCleanup(5)

cdat = clean_data.cl_fraction(dat,'cell_area')
print(cdat.head(2))
print('removed ', dat.shape[0] - cdat.shape[0], ' cells out of ', dat.shape[0], ' based on cell area')


cdat['CD16'] = 'negative'
cdat.loc[cdat['MeanCD16'] >= 0.0049, 'CD16'] = 'positive'


#save single-cell data with outliers removed 
cdat.to_csv(output_folder+'single_cell_cleaned/'+exp_name+'_cell_clean.csv')



Data location:  /home/siina/python_scripts/moncyte_analysis/analysis_tools/single_cell_test/cellp_results/ME00043_P1_Cells.csv
Experiment name:  ME00043_P1
Run name:  ME0004
Plate layout:  /home/siina/python_scripts/moncyte_analysis/analysis_tools/single_cell_test/layouts/ME00043_P1_layout.csv
       well Names
0       NaN     A
1    r01c01   NaN
2    r01c02   NaN
3    r01c03   NaN
4    r01c04   NaN
..      ...   ...
395  r16c20   NaN
396  r16c21   NaN
397  r16c22   NaN
398  r16c23   NaN
399  r16c24   NaN

[400 rows x 2 columns]
       Metadata_Filename  cell_area  LDOrg       IntLD    IntCD16    MeanLD  \
0  r03c18f1-ch1_MIP.tiff       5299      0  133.492928  22.680843  0.025192   
1  r03c18f1-ch1_MIP.tiff       4109      0   91.865446  10.166278  0.022357   

   MeanCD16     MedLD   MedCD16  Size_LDOrg  MeanIntLD_LDOrg  \
0  0.004280  0.025193  0.004334         NaN              NaN   
1  0.002474  0.023331  0.002457         NaN              NaN   

   MeanIntCD16_LDOrg  MeanMeanLD_L